In [7]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv
import betfairlightweight
from betfairlightweight import filters
import ast

In [2]:
# Load environment variables
load_dotenv()

# Betfair API setup
bf_usr = os.getenv("BF_LOGIN")
bf_pass = os.getenv("BF_PASS")
bf_api = os.getenv("BF_API_KEY")
bf_certs_path = '../certs/'

# Initialize Betfair client
client = betfairlightweight.APIClient(bf_usr, bf_pass, app_key=bf_api, certs=bf_certs_path)
client.login()

<LoginResource>

In [8]:
def get_polymarket_data():
    r = requests.get("https://gamma-api.polymarket.com/events?closed=false")
    response = r.json()

    market_data = []
    for market in response:
        overall_data = {
            'Market ID': market.get('id', 'N/A'),
            'Title': market.get('title', 'N/A'),
            'End Date': market.get('endDate', 'N/A'),
            'Overall Liquidity': float(market.get('liquidity', 0)),
            'Overall Volume': float(market.get('volume', 0)),
            'Overall Volume 24hr': float(market.get('volume24hr', 0)),
        }
        for bet in market.get('markets', []):
            outcome_prices_str = bet.get('outcomePrices', '["N/A", "N/A"]')
            try:
                outcome_prices = ast.literal_eval(outcome_prices_str)
                yes_price = float(outcome_prices[0]) if len(outcome_prices) > 0 else None
                no_price = float(outcome_prices[1]) if len(outcome_prices) > 1 else None
            except (ValueError, SyntaxError):
                yes_price = None
                no_price = None
                
            bet_data = {
                'Bet ID': bet.get('id'),
                'Bet Question': bet.get('question', 'N/A'),
                'Bet Liquidity': float(bet.get('liquidity', 0)),
                'Bet Volume': float(bet.get('volume', 0)),
                'Bet Volume 24hr': float(bet.get('volume24hr', 0)),
                'Yes Price': yes_price,
                'No Price': no_price,
            }
            combined_data = {**overall_data, **bet_data}
            market_data.append(combined_data)

    return pd.DataFrame(market_data)

def get_betfair_data():
    market_filter = filters.market_filter(event_type_ids=['2378961'])  # Politics event type
    market_catalogues = client.betting.list_market_catalogue(
        filter=market_filter,
        max_results=100,
        market_projection=['RUNNER_DESCRIPTION']
    )

    market_ids = [market.market_id for market in market_catalogues]
    price_filter = filters.price_projection(price_data=['EX_BEST_OFFERS'])

    betfair_data = []
    for market in market_catalogues:
        market_books = client.betting.list_market_book(
            market_ids=[market.market_id],
            price_projection=price_filter
        )
        if market_books:
            market_book = market_books[0]
            for runner in market_book.runners:
                runner_data = {
                    'Market ID': market.market_id,
                    'Market Name': market.market_name,
                    'Selection ID': runner.selection_id,
                    'Selection Name': next((r.runner_name for r in market.runners if r.selection_id == runner.selection_id), 'Unknown'),
                    'Best Back Price': runner.ex.available_to_back[0].price if runner.ex.available_to_back else None,
                    'Best Back Size': runner.ex.available_to_back[0].size if runner.ex.available_to_back else None,
                    'Best Lay Price': runner.ex.available_to_lay[0].price if runner.ex.available_to_lay else None,
                    'Best Lay Size': runner.ex.available_to_lay[0].size if runner.ex.available_to_lay else None,
                }
                betfair_data.append(runner_data)

    return pd.DataFrame(betfair_data)


In [15]:
def find_arbitrage(polymarket_df, betfair_df):
    # Load market and selection mappings
    market_mapping = pd.read_csv('../data/market_mapping.csv')
    selection_mapping = pd.read_csv('../data/selection_mapping.csv')

    # Convert IDs to strings in all DataFrames
    polymarket_df['Market ID'] = polymarket_df['Market ID'].astype(str)
    polymarket_df['Bet ID'] = polymarket_df['Bet ID'].astype(str)
    betfair_df['Market ID'] = betfair_df['Market ID'].astype(str)
    betfair_df['Selection ID'] = betfair_df['Selection ID'].astype(str)
    market_mapping['polymarket_market_id'] = market_mapping['polymarket_market_id'].astype(str)
    market_mapping['betfair_market_id'] = market_mapping['betfair_market_id'].astype(str)
    selection_mapping['polymarket_bet_id'] = selection_mapping['polymarket_bet_id'].astype(str)
    selection_mapping['betfair_selection_id'] = selection_mapping['betfair_selection_id'].astype(str)

    # Apply mappings
    polymarket_df = pd.merge(polymarket_df, market_mapping, left_on='Market ID', right_on='polymarket_market_id', how='left')
    polymarket_df = pd.merge(polymarket_df, selection_mapping, left_on='Bet ID', right_on='polymarket_bet_id', how='left')

    # Merge Polymarket and Betfair data
    merged_df = pd.merge(
        polymarket_df,
        betfair_df,
        left_on=['betfair_market_id', 'betfair_selection_id'],
        right_on=['Market ID', 'Selection ID'],
        how='inner',
        suffixes=('_poly', '_bet')
    )

    arbitrage_opportunities = []
    for _, row in merged_df.iterrows():
        # Check arbitrage between Polymarket Yes price and Betfair Lay price
        if row['Best Lay Price'] and row['Yes Price'] and row['Best Lay Price'] > 1 / row['Yes Price']:
            arbitrage_opportunities.append({
                'Market': row['Title'],
                'Selection': row['Selection Name'],
                'Type': 'Polymarket Yes vs Betfair Lay',
                'Polymarket Odds': 1 / row['Yes Price'],
                'Betfair Odds': row['Best Lay Price'],
                'Profit %': (row['Best Lay Price'] / (1 / row['Yes Price']) - 1) * 100
            })

        # Check arbitrage between Polymarket No price and Betfair Back price
        if row['Best Back Price'] and row['No Price'] and row['Best Back Price'] > 1 / (1 - row['No Price']):
            arbitrage_opportunities.append({
                'Market': row['Title'],
                'Selection': row['Selection Name'],
                'Type': 'Polymarket No vs Betfair Back',
                'Polymarket Odds': 1 / (1 - row['No Price']),
                'Betfair Odds': row['Best Back Price'],
                'Profit %': (row['Best Back Price'] / (1 / (1 - row['No Price'])) - 1) * 100
            })

    return pd.DataFrame(merged_df)

In [17]:
# Main execution
#polymarket_data = get_polymarket_data()
#betfair_data = get_betfair_data()
arbitrage_opportunities = find_arbitrage(polymarket_data, betfair_data)

print(arbitrage_opportunities)

  Market ID_poly                                  Title              End Date  \
0         903193      Presidential Election Winner 2024  2024-11-04T12:00:00Z   
1         903193      Presidential Election Winner 2024  2024-11-04T12:00:00Z   
2         903193      Presidential Election Winner 2024  2024-11-04T12:00:00Z   
3         903665  Michigan Presidential Election Winner  2024-11-04T12:00:00Z   

   Overall Liquidity  Overall Volume  Overall Volume 24hr  Bet ID  \
0       5.107804e+08    2.173560e+09         3.160379e+07  253591   
1       5.107804e+08    2.173560e+09         3.160379e+07  253592   
2       5.107804e+08    2.173560e+09         3.160379e+07  253597   
3       1.243913e+06    1.839669e+07         3.213801e+05  255143   

                                        Bet Question  Bet Liquidity  \
0  Will Donald Trump win the 2024 US Presidential...   1.923820e+06   
1  Will Joe Biden win the 2024 US Presidential El...   2.665108e+07   
2  Will Kamala Harris win the 2024 

In [ ]:
polymarket_data.to_csv('../output/polymarket_data.csv', index=False)
betfair_data.to_csv('../output/betfair_data.csv', index=False)